<a href="https://colab.research.google.com/github/Kikkas01/Python-for-analytics/blob/Lesson3/%D0%A3%D1%80%D0%BE%D0%BA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 3. Задание 1
Использовать датасет с текущего урока и построить сегментированную воронку
конверсии пользователей интернет-магазина по полу.

In [60]:
import pandas as pd

In [61]:
user_table = pd.read_csv('user_table.csv')

In [62]:
home_page = pd.read_csv('home_page_table.csv')

In [63]:
search_page = pd.read_csv('search_page_table.csv')

In [64]:
payment_page = pd.read_csv('payment_page_table.csv')

In [65]:
payment_confirmation = pd.read_csv('payment_confirmation_table.csv')

In [66]:
home_page = home_page.rename(columns = {'page':'home_page'})
search_page = search_page.rename(columns = {'page':'search_page'})
payment_page = payment_page.rename(columns = {'page':'payment_page'})
payment_confirmation = payment_confirmation.rename(columns = {'page':'payment_confirmation'})

In [67]:
df = user_table.merge(home_page, on = 'user_id', how = 'left').merge(search_page, on = 'user_id', how = 'left') \
.merge(payment_page, on = 'user_id', how = 'left').merge(payment_confirmation, on = 'user_id', how = 'left')
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,NaN,NaN,NaN
1,756838,2015-01-13,Desktop,Male,home_page,NaN,NaN,NaN
2,568983,2015-04-09,Desktop,Male,home_page,search_page,NaN,NaN
3,190794,2015-02-18,Desktop,Female,home_page,search_page,NaN,NaN
4,537909,2015-01-15,Desktop,Male,home_page,NaN,NaN,NaN


In [68]:
df['home_page'] = df['home_page'].apply(lambda x: 1 if x == 'home_page' else 0)
df['search_page'] = df['search_page'].apply(lambda x: 1 if x == 'search_page' else 0)
df['payment_page'] = df['payment_page'].apply(lambda x: 1 if x == 'payment_page' else 0)
df['payment_confirmation'] = df['payment_confirmation'].apply(lambda x: 1 if x == 'payment_confirmation_page' else 0)

df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


In [69]:
summary = df[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().reset_index()
summary.head()

,index,0
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation,452


In [70]:
summary= summary.rename(columns = {'index':'stage', 0:'number'})
summary.head()

,stage,number
0,home_page,90400
1,search_page,45200
2,payment_page,6030
3,payment_confirmation,452


In [71]:
dfg = df.groupby('sex')[['home_page', 'search_page', 'payment_page', 'payment_confirmation']].sum().unstack('sex').unstack('sex').reset_index()
dfg.head()

sex,index,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [72]:
dfg.columns.name = None
dfg = dfg.rename(columns = {'index':'Action'})
dfg.head()

,Action,Female,Male
0,home_page,45075,45325
1,search_page,22676,22524
2,payment_page,3100,2930
3,payment_confirmation,241,211


In [73]:
from plotly import graph_objects as go

In [74]:
fig = go.Figure()

fig.add_trace(go.Funnel(
    y = dfg['Action'],
    x = dfg['Female'],
    name = 'Female',
    constraintext='outside',
    textposition = 'inside',
    textinfo = 'value+percent previous'
    ))

fig.add_trace(go.Funnel(
    y = dfg['Action'],
    x = dfg['Male'],
    name = 'Male',
    textposition = 'auto',
    textinfo = 'value+percent previous'
   ))

fig.update_layout(
    title = 'Воронка конверсии по полу покупателей',
    title_x=0.5,
    autosize=False,
    width=1200,
    height=600)

fig.show()

# Урок 3. Задание 2
Создать 2 новые фичи на основе колонки “date”: месяц и день недели

In [75]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1


In [76]:
# Долго бился, чтобы сделать названия дней, а не цифры, сначала было закомментированное
# ниже решение, но потом осенило, что можно сделать через lambda:
df['day_of_week'] = df['date'].dt.dayofweek 
df['day_of_week'] = df['day_of_week'].apply(lambda x: 
                                            'Monday' if x == 0
                                            else 
                                            'Tuesday' if x == 1
                                            else
                                            'Wednesday' if x == 2
                                            else
                                            'Thursday' if x == 3
                                            else
                                            'Friday' if x == 4
                                            else
                                            'Saturday' if x == 5
                                            else
                                            'Sunday' if x == 6
                                            else 'no data'
                                            )


# совсем не изящное, но работающее решение №2:
# temp_day = df['date'].dt.dayofweek
# days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# temp_day_1 = []
# for i in temp_day:
#   if i == 0:
#     temp_day_1.append(days[0])
#   elif i == 1:
#     temp_day_1.append(days[1])
#   elif i == 2:
#     temp_day_1.append(days[2])
#   elif i == 3:
#     temp_day_1.append(days[3])
#   elif i == 4:
#     temp_day_1.append(days[4])  
#   elif i == 5:
#     temp_day_1.append(days[5])
#   elif i == 6:
#     temp_day_1.append(days[6])

# df['day_of_week_hard'] = temp_day_1

df.head(10)

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation,month,day_of_week
0,450007,2015-02-28,Desktop,Female,1,0,0,0,2,Saturday
1,756838,2015-01-13,Desktop,Male,1,0,0,0,1,Tuesday
2,568983,2015-04-09,Desktop,Male,1,1,0,0,4,Thursday
3,190794,2015-02-18,Desktop,Female,1,1,0,0,2,Wednesday
4,537909,2015-01-15,Desktop,Male,1,0,0,0,1,Thursday
5,993454,2015-03-03,Desktop,Male,1,0,0,0,3,Tuesday
6,377150,2015-04-15,Desktop,Female,1,1,0,0,4,Wednesday
7,137258,2015-01-27,Mobile,Male,1,0,0,0,1,Tuesday
8,608249,2015-04-12,Desktop,Female,1,0,0,0,4,Sunday
9,615615,2015-02-24,Mobile,Female,1,1,0,0,2,Tuesday


# Урок 3. Задание 3
Определить самый топовый по продажам месяц и день недели с помощью
базовых методов агрегации (sum или count).

In [77]:
df['month'].nunique()

4

In [78]:
df['payment_confirmation'].count()

90400

In [79]:
df.groupby('month')['payment_confirmation'].count()
# Получилось, что все 4 месяца одинаковы по количеству продаж и топового нет

month
1    22600
2    22600
3    22600
4    22600
Name: payment_confirmation, dtype: int64

In [80]:
df.groupby('day_of_week')['payment_confirmation'].count()
# Максимальное количество продаж было в четверг. Пробовал добавлять функцию 
# max() в конце, чтобы остался только топ, но пропадает название дня, не смог победить

day_of_week
Friday       12750
Monday       12955
Saturday     12801
Sunday       12922
Thursday     13444
Tuesday      12697
Wednesday    12831
Name: payment_confirmation, dtype: int64

# Урок 3. Задание 4
Выяснить, в какой день недели лучше покупают женщины, а в какой мужчины с
помощью одного из методов построения сводных таблиц (groupby, pivot_table или
crosstab).

In [81]:
df = df.rename(columns={'day_of_week':'day'}) 
df.groupby(['day', 'sex'])['payment_confirmation'].count().unstack('day')
# решил еще переименовать столбец, чтобы вывод информации был покороче

day,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
sex,,,,,,,
Female,6328,6400,6344,6506,6728,6410,6359
Male,6422,6555,6457,6416,6716,6287,6472
